In [232]:
import pyarrow as pa
from pyarrow import json
from pyarrow import compute as pc
from pyarrow import parquet
import polars as pl
from pathlib import Path

In [233]:
def clean_date(col):
    for remove in [
        r"\.$",
        r"\]$", 
        "^[c\[]",
        r"\?$",
        "(Heisei|Min guo) [0-9]+ \[",
        "-$",
        "^[c\[]",
    ]:
        col = col.str.replace(remove, "")
    return col.cast(pl.Int32, strict=False).alias("year")


def singleton(fname, fs = None, bail = False):
    """
    fname: A field, like "240$b"
    fs: a pyarrow table.
    
    returns: a tidy table reflecting the 
    """
    try:
        f, sub = fname.split("$")
    except ValueError:
        f = fname
        sub = None
    if fs is None:
        all = []
        for fin in Path("parquet").glob("*.parquet"):
            fs = parquet.read_table(fin.open('rb'), columns = ['001', f])
            all.append(singleton(fname, fs))
        return pl.concat(all)
    if pa.types.is_list(fs[f].combine_chunks().type):
        field = pc.list_flatten(fs[f].combine_chunks())
        ids = fs['001'].take(pc.list_parent_indices(fs[f])).combine_chunks()
    else:
        field = fs[f].combine_chunks()
        ids = fs['001']
    if sub is None:
        return pl.from_arrow(pa.table({
        f'{fname}': field,
        f'lccn': ids
    }))
    loc = field.field(sub)
    if pa.types.is_list(loc.type):
        ixes = pc.list_parent_indices(loc)
        ids = ids.take(ixes)
        loc = pc.list_flatten(loc)
    return pl.from_arrow(pa.table({
        f'{f}${sub}': loc,
        f'lccn': ids
    }))


In [241]:
additional = singleton("710$a")

In [263]:
additional.filter(pl.col("710$a")=="Copyright Paperback Collection (Library of Congress)").join(sizes, on="lccn").groupby("height").count().sort("height").filter(pl.col("count") > 50)

height,count
i32,u32
null,701
16,65
17,13732
18,45764
19,1174
20,3129
21,205
22,133


In [234]:
titles = singleton("245$a", bail = False)

In [235]:
%%time

places = singleton("260$a")

places.groupby('260$a').count().sort('count', reverse = True).head(10)

CPU times: user 3.63 s, sys: 570 ms, total: 4.2 s
Wall time: 4.24 s


260$a,count
str,u32
"""New York :""",881798
"""New York,""",330622
"""London,""",190088
"""Paris :""",182845
"""Moskva :""",169123
"""London :""",166368
"""Tōkyō :""",152686
"""Boston :""",96270
"""Washington :""",81787


In [236]:
def clean_year(fs):
    year = singleton('260$c', fs)\
       .select(["lccn", '260$c', clean_date(pl.col('260$c'))])#.groupby('260$c').count().sort('count', reverse = True)
    year2 = singleton('264$c', fs)\
       .select(["lccn", '264$c', clean_date(pl.col('264$c'))])#.groupby('260$c').count().sort('count', reverse = True)
    return year.join(year2, on="lccn", how="outer").select([
        "lccn", pl.when(pl.col("year").is_null()).then(pl.col('year_right')).otherwise(pl.col('year')).alias('year')])\
    .filter(pl.col('year').is_not_null())\
    .groupby('lccn').agg(pl.col('year').first())

 

In [237]:
def clean_size(size, posthaste = False):
    if posthaste == False:
        size = singleton("300$c", size).select(['lccn', pl.col("300$c").str.replace("\.$", "")])
    size = size.select([
        pl.all(),
        pl.col('300$c')\
                 .str.replace("X", "x")\
                 .str.replace(r" cm\. \+$", " cm")\
                 .str.replace(" \((8v|4t)o\)$", "")\
                 .str.replace(r"^(\d+) x (\d+) cm\.?$", "\1 cm")\
                 .str.replace(r"\.$", "")\
                 .str.replace(" cm$", "").cast(pl.Int32, strict = False).alias("height")])
    return size

In [238]:
def run_batch(fname, bail = False):
    fs = parquet.read_table(fname, columns=["050", "300", "001", "260", "264"])
    classes = singleton("050$a", fs).select([pl.col("lccn"), pl.col("050$a").str.replace("[^A-Z].+", "").alias("lc1")])

    size = clean_size(fs)
    # https://web.library.yale.edu/cataloging/CIP/editing-300c
    # the width of the volume is either less than half the height or greater than the height, give in the form <height> x <width>

    if bail:
        return size, classes, year, year2
    
    size =  size.filter(pl.col('height').is_not_null())
    return year.join(size, on = "lccn").join(classes, on = "lccn")\
      .filter(pl.col('year') < 2025)\
      .filter(pl.col('year') > 1750)\
      .filter(pl.col('height') < 60)\
      .filter(pl.col('height') > 10)\
      .groupby(["year", "height", "lc1"]).count()

In [239]:
counts = []
for p in Path("parquet").glob("*.parquet"):
    print(p, end = "\r")
    b = run_batch(p)
    if b.shape[0] < 10_000:
        break
    counts.append(b)

NameError: name 'year' is not defined

In [145]:
langs = singleton("008").select([pl.col("008").str.slice(35, 3).alias("lang"), pl.col("lccn")])

In [151]:
publishers = singleton("260$b")

In [61]:
sizes = singleton("300$c")
sizes = clean_size(sizes, True)

In [194]:
publishers_size = publishers.join(sizes, on="lccn").join(langs, on="lccn").filter(pl.col("lang")=="eng")



In [204]:
publishers_size.groupby([pl.col("260$b").str.replace(",$", "").str.replace(" Books$", "").alias("publisher")])\
.agg([pl.col("height").median(), pl.count()]).sort("count", reverse = True)\
.filter(pl.col("count") > 100).groupby("height").head(10).sort('height').filter(pl.col("height") < 19)

height,publisher,count
f64,str,u32
13.0,"""Haldeman-Julius Co.""",512
13.0,"""Haldeman-Julius Publications""",475
15.0,"""Lonely Planet""",151
15.0,"""American Sunday-School Union, no. 146 Chestnut Street""",126
16.0,"""CandyCane Press""",177
16.0,"""American Sunday-School Union""",114
17.0,"""Harlequin""",6177
17.0,"""Novosti Press Agency Pub. House""",1308
17.0,"""Steeple Hill""",771


In [205]:
crossjoin = titles.lazy().join(sizes.lazy(), on = "lccn").join(langs.lazy(), on="lccn").collect()

In [114]:
crossjoin

(11534131, 2)

In [210]:
import random
crossjoin.filter(pl.col("height") == 17).filter(pl.col("lang").str.contains("eng")).sample(n=10, seed=random.randint(0, 10000))\
.join(publishers, on = "lccn")

245$a,lccn,300$c,height,lang,260$b
str,str,str,i32,str,str
"""Josie /""",""" 2002553387""","""17 cm.""",17,"""eng""","""Scholastic Inc.,"""
"""A matter of time /""",""" 2002559885""","""17 cm.""",17,"""eng""","""Jove Books,"""
"""Never mind the face, or, The cousin's visit /""",""" 2005576189""","""17 cm.""",17,"""eng""","""C. Scribner,"""
"""City of passion /""",""" 88091583 ""","""17 cm.""",17,"""eng""","""Avon Books,"""
"""Social justice in Israel /""",""" 78111249 ""","""17 cm.""",17,"""eng""","""World Wizo, Dept. of Organisation and Education,"""
"""A book of poems,""",""" 00003873 ""","""17 cm.""",17,"""eng""","""Press of Stone Ptg. & Mfg. Co.,"""
"""The Italian count's baby /""",""" 2009367580""","""17 cm.""",17,"""eng""","""Harlequin,"""
"""Living a charmed life :""",""" 2008051466""","""17 cm.""",17,"""eng""","""HarperOne,"""
"""The social record of Christianity,""",""" 35013610 ""","""17 cm.""",17,"""eng""","""Watts & co."""


In [224]:
crossjoin.join(publishers.filter(pl.col("260$b") == "HarperOne,"), on="lccn")\
    .filter(pl.col("height") == 24).shape\
#    .filter(pl.col("lang").str.contains("eng")).sample(n=10, seed=random.randint(0, 10000))


(142, 6)

In [231]:
crossjoin.join(publishers.filter(pl.col("260$b") == "HarperCollins,"), on="lccn").groupby("height").count().sort("height").head(15)

height,count
i32,u32
null,280
11,5
12,24
13,3
14,4
15,8
16,22
17,43
18,199


In [ ]:
d = pl.concat(counts).groupby(["year", "height", "lc1"]).sum()\
  .sort("count").to_csv("prelim.csv")

In [ ]:
y1.join(y2, on="lccn", how="outer").select([
    "lccn", pl.when(pl.col("year").is_null()).then(pl.col('year_right')).otherwise(pl.col('year')).alias('year')])\
.filter(pl.col('year').is_not_null())\
.groupby('lccn').agg(pl.col('year').first()).shape

In [ ]:
#pl.DataFrame({'c': pl.from_arrow().flatten()[0])}).groupby('c')\
#  .count().sort(['count'])
pl.from_arrow(pa.Table.from_batches([pa.RecordBatch.from_struct_array(pc.list_flatten(fs['300'].combine_chunks()).flatten()[1].flatten().value_counts())])).sort("counts")


In [ ]:
d.groupby('year').sum().sort("count", reverse = True)

In [ ]:
[c['count'].sum() for c in counts]